In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.core import Experiment
import joblib

In [2]:
from train import clean_data

In [3]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project")

# Get the "default" workspace configuration that I did not specifically create
ws = Workspace.from_config()

# Create the experiment and name it exp-udacity-project
exp = Experiment(workspace=ws, name="exp-udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-152448
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-152448


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Use https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py as a reference
#
# Name the cluster GPUCluster
cluster_name = "CPUCluster"

# See if it already exists
# this ofcourse helps if you are reunning the notebook from teh start and do not
# need to recreate the compute cluster
try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    # No exception thrown - Found it - use it below
except ComputeTargetException:
    # Did not find the compute target - will need to create one
    # Specify the compute cluster configuration first
    # See https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target
    # and after clicking on Dv2
    # see https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target

    # CPU cluster instance - from the Microsoft recommended options
    cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS3_v2',
                                                            max_nodes=4,
                                                            description='CPU Compute Cluster created programatically')

    # Next, create the cluster
    compute_cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

# We have a compute cluster - either newly created - or created earlier
# We may wait for the create operation to complete
compute_cluster.wait_for_completion(show_output=True)


Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig
# Specify parameter sampler
# ps = ### YOUR CODE HERE ###

# Prepare to use teh Azure ML HyperDrive operational workflow
# Essentially, HyperDrive will
# - invoke the training script multiple times, each time with differnet ML hyper parameters (--C and --max_iter in this case)
# - the script will log the hyper parameters, the accuracy and the model (all instrumentation) for each invocation
#
# THe workflow is to
# - prepare the run configuration
# - prepare all the other parameters
# - "Submit" the experiment and let the HyperDrive pipeline do it's things
# - Determine the best run
# - Retrieve and register the model from the best run
# 
# A lot of rederence information may be found at 
# "Hyperparameter tuning a model with Azure Machine Learning" - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

# import normal and choice as they are used below in the parameter sampling

from azureml.train.hyperdrive.parameter_expressions import normal, choice

# from train.py - we know that the parameters are --C and --max_iter
# train.py uses the scikit-learn LogisticRegression model

ps = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10),
    "--max_iter": choice(100, 200, 300, 400, 500),
})
# Specify a Policy
#policy = ### YOUR CODE HERE ###

# BanditPolicy is a type of EarlyTerminationPolicy
# More info can be found at 
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)


# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###


# We need to specify/provide an environment
# We could create our own environment with corresponding yaml file
# We may also provide an environment 'out-of-the-box' from one of the many environments provided

curated_env_name = 'AzureML-Tutorial'
curated_env = Environment.get(workspace=ws, name=curated_env_name)

# SKLearn is deprecated - instead we specity the run configuration using ScriptRunConfig
# It is necessary to pass the directory, the python script, the compute cluster and the environment

run_config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_cluster,
    environment=curated_env,
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    policy=policy,
    run_config=run_config,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
# from azureml.core import Experiment 

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d402ec09-cea3-42f6-82f2-78885cb3d631
Web View: https://ml.azure.com/runs/HD_d402ec09-cea3-42f6-82f2-78885cb3d631?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-152448/workspaces/quick-starts-ws-152448&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-07-25T06:55:42.023550][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2021-07-25T06:55:41.572878][API][INFO]Experiment created<END>\n""<START>[2021-07-25T06:55:42.291566][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d402ec09-cea3-42f6-82f2-78885cb3d631
Web View: https://ml.azure.com/runs/HD_d402ec09-cea3-42f6-82f2-78885cb3d631?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-152448/workspaces/quick-starts-ws-152448&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



In [7]:
# Get your best run and save the model from that run.


### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the model hyperparameter values corresponding to the best run
print(f'********** {best_hyperdrive_run.get_details()["runDefinition"]["arguments"]}')
# print(best_hyperdrive_run.get_details())

# Get the accuracy corresponsding to the best run
print(f'********** Best HyperDrive run Accuracy: {best_hyperdrive_run.get_metrics()["Accuracy"]}')



********** ['--C', '0.1', '--max_iter', '100']
********** Best HyperDrive run Accuracy: 0.9153262518968134


In [9]:
# Save and download the model
print(f'The file name for the model is: {best_hyperdrive_run.get_file_names()[-1]}')

# download the file in the notebook context. In this case, under Users/**user**/outputs/
best_hyperdrive_run.download_file(name=best_hyperdrive_run.get_file_names()[-1], output_file_path='./outputs/model.joblib')

The file name for the model is: outputs/model.joblib


In [10]:
# load the model to ensure it all looks good
joblib.load('./outputs/model.joblib')

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
# Vrify from above that --C  and --max_iter are our model values

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
# Create the Azure ML dataset from the preferred source
# Note that thsi source is the same as used in train.py

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [13]:
AUTOML_DATA_DIR = 'automl_data'

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
# clean the data - function already introcuced in train.py
x, y = clean_data(ds)

# Note that the automl config does not call for a validation data set; therefore all
# of the data would be used as training data
#
# Add x and y together
# save the new df to disk as a csv
# Upload to a datastore
# 
# load from datastore as an Azure TabularDataSet

# Add two pandas dataframes togethere
x['y'] = y


# save and reload the clean data so that Azure ML can use it
# See https://stackoverflow.com/questions/60380154/upload-dataframe-as-dataset-in-azure-machine-learning

# To be able to load to datastore - the data needs to be in a folder.
# Thus first create the directory

# Create a directory if "my_data" not in os.listdir():
if AUTOML_DATA_DIR not in os.listdir():
    os.mkdir('./' + AUTOML_DATA_DIR)


# now save x to disk
x.to_csv(AUTOML_DATA_DIR + '/clean_data.csv')

# upload the file to the default datastore
datastore = ws.get_default_datastore()

datastore.upload(src_dir=AUTOML_DATA_DIR, target_path=AUTOML_DATA_DIR)

Uploading an estimated of 1 files
Uploading automl_data/clean_data.csv
Uploaded automl_data/clean_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_96f9420c49db44d3affef5e93d4c8e39

In [14]:
# Now Create the dataset that will later be used for the ML Pipeline

clean_ds = TabularDatasetFactory.from_delimited_files(datastore.path(AUTOML_DATA_DIR + '/clean_data.csv'))

In [15]:
from azureml.train.automl import AutoMLConfig
import logging

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

#TODO here
# From examples here - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# this dataset discusses wehther a user may be interested in a term deosit or not
# It thus becomes a classification problem
# The metric used is accuracy - the same that is used by the LogisticRegression in train.py
# 
# 
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "max_concurrent_iterations" : 4,
    "max_cores_per_iteration" : -1,
    "n_cross_validations" : 2,
    "primary_metric" : 'accuracy',
    "verbosity" : logging.INFO,
}

# Provide the remainder of the settings/configuration
# Note that we are not providing a validation data set - and we may need to
# 
automl_config = AutoMLConfig(
    compute_target = compute_cluster,
    task='classification',
    training_data=clean_ds,
    label_column_name='y',
    **automl_settings)


In [17]:
# Submit your automl run

### YOUR CODE HERE ###
# From https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train again
#
automl_run = exp.submit(config=automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == 'Completed')

Submitting remote run.
No run_configuration provided, running on CPUCluster with default configuration
Running on remote compute: CPUCluster


Experiment,Id,Type,Status,Details Page,Docs Page
exp-udacity-project,AutoML_11ebc447-195b-4325-987e-cb90a0755d2e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

Experiment,Id,Type,Status,Details Page,Docs Page
exp-udacity-project,AutoML_11ebc447-195b-4325-987e-cb90a0755d2e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:01:27       0.9134    0.9134
         1   MaxAbsScaler XGBoostClassifier                 0:00:56       0.9130    0.9134
         2   MaxAbsScaler RandomForest                      0:00:48       0.8959    0.9134
         3   MaxAbsScaler RandomForest                      0:01:23       0.8880    0.9134
         4   MaxAbsScaler RandomForest                      0:00:47       0.7980    0.9134
         5   MaxAbsScaler RandomForest                      0:00:46       0.7458    0.9134
         6   SparseNormalizer XGBoostClassifier             0:00:51       0.9123    0.9134
         7   SparseN

AttributeError: 'AutoMLRun' object has no attribute 'getstatus'

In [18]:
automl_run.get_status()

'Completed'

In [19]:
# define print_model - from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()  

In [30]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run, automl_best_model = automl_run.get_output()

# print the metrics:
automl_best_run_metrics = automl_best_run.get_metrics()

print(f'********** Best AutoML accuracy: {automl_best_run_metrics.get("accuracy")}')

print(f'********** printing Best AutoML run:\n{automl_best_run}\n\nPrinting model:')

print_model(automl_best_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


********** Best AutoML accuracy: 0.9153262518968134
********** printing Best AutoML run:
Run(Experiment: exp-udacity-project,
Id: AutoML_11ebc447-195b-4325-987e-cb90a0755d2e_36,
Type: azureml.scriptrun,
Status: Completed)

Printing model:
datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/nb-compute/code/Users/odl_user_152448'}

prefittedsoftvotingclassifier
{'estimators': ['0', '1', '32', '6', '20', '5', '8', '33'],
 'weights': [0.2, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1]}

0 - maxabsscaler
{'copy': True}

0 - lightgbmclassifier
{'min_data_in_leaf': 20,
 'n_jobs': -1,
 'problem_info': ProblemInfo(
    dataset_samples=32950,
    dataset_features=123,
    dataset_classes=None,
  

In [27]:
# Save the best model
automl_best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_374691d9b023be560870a3e37dbac630074a71c43543685859a4f2ba3c1661e3_d.txt',
 'azureml-logs/65_job_prep-tvmps_374691d9b023be560870a3e37dbac630074a71c43543685859a4f2ba3c1661e3_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_374691d9b023be560870a3e37dbac630074a71c43543685859a4f2ba3c1661e3_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'explanation/c7973a1d/classes.interpret.json',
 'explanation/c7973a1d/eval_data_viz.interpret.json',
 'explanation/c7973a1d/expected_values.interpret.json',
 'explanation/c7973a1d/features.interpret.json',
 'explanation/c7973a1d/global_names/0.interpret.json',
 'explanation/c7973a1d/global_rank/0.interpret.json',
 'explanation/c7973a1d/global_values/0.interpret.json',
 'explanation/c7973a1d/local_importance_values.interpret.json',
 'explanation/c7973a1d/per_class_names/0.interpret.json',
 'explanati

In [28]:
# Download the best AutoML model to the nbotebook context
joblib.dump(automl_best_model, './outputs/automl_model.joblib')

['./outputs/automl_model.joblib']

In [29]:
# Save and register the model
automl_best_run.register_model(model_name='automl_registered_best_model.pkl', model_path='./outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-152448', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-152448'), name=automl_registered_best_model.pkl, id=automl_registered_best_model.pkl:2, version=2, tags={}, properties={})